# Feed-Forward Network
Builder & Example for a feed-forward fully-connected neural network with TensorFlow
### Imports

In [1]:
import tensorflow as tf
import tensorflow.python.platform
import math
import numpy
import os
import sys

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from read_data import DataSet

### Flags
Basic model parameters as external flags.

In [3]:
LEARNING_RATE = 0.01
TRAIN_DIR = 'data'
TEST_DIR = 'data'
MAX_EPOCHS = 2000
BATCH_SIZE = 100

### Input & Output Label Placeholders
Returns two placeholders for the input/labels constructed for a given batch size... 

In [4]:
def get_input_label_placeholder(input_dim):
    input_placeholder = tf.placeholder(tf.float32, shape=(BATCH_SIZE,input_dim))
    label_placeholder = tf.placeholder(tf.float32, shape=(BATCH_SIZE))
    return input_placeholder, label_placeholder

### Input Feed Dictionary 
Constructions a feed_dict to use with the SGD optimizer

In [5]:
def fill_feed_dict(data_set, input_placeholder, label_placeholder):
    input_feed, label_feed = data_set.next_batch(BATCH_SIZE)
    feed_dict = {
        input_placeholder: input_feed,
        label_placeholder: label_feed,
    }
    return feed_dict

### Weights
Returns a tensor with the desired shape (array) and randomly initializes weights using seed value

In [6]:
def get_weight_tensor(input_dim, output_dim):
    shape = [input_dim, output_dim]
    std = 1.0 / math.sqrt(float(input_dim))
    rand_init_values = tf.truncated_normal(shape, stddev=std)
    return tf.Variable(rand_init_values, name='weights')

### Bias
Returns a tensor with the desired shape (array) and initializes with zeros

In [7]:
def get_bias_tensor(dim):
    zero_init_values = tf.zeros(dim)
    return tf.Variable(zero_init_values, name='biases')

### Layer Input
Returns tensorflow opearation for the multiplication of the weights * inputs + bias

In [8]:
def get_mult_op(inputs, weights, bias):
    return tf.matmul(inputs, weights) + bias

### Activation Function
Returns a tensorflow activation function based on the name passed to the method

In [9]:
def get_activation(layer, type='sigmoid'):
    if type is 'sigmoid': return tf.nn.sigmoid(layer)
    elif type is 'relu': return tf.nn.relu(layer)
    elif type is 'softmax': return tf.nn.softmax(layer)
    elif type is 'tanh': return tf.nn.tanh(layer)
    elif type is 'linear': return layer
    else: return layer

### Build Layer
Returns a new layer constructed from the inputs with the specified name scope

In [10]:
def build_layer(name, input_vec, input_dim, output_dim, activ='sigmoid'):
    # size of the input
    with tf.name_scope(name):
        # weight matrix
        weights = get_weight_tensor(input_dim, output_dim)
        # bias vector
        bias = get_bias_tensor(output_dim)
        # weighted combination of inputs
        mult = get_mult_op(input_vec, weights, bias)
        # activation function
        output = get_activation(mult, activ)
    return output 

### Network Constructor
Given a shape and an input placeholder, builds an FF network with a specified activation and output types

In [11]:
def construct_network(input_vec, shape, num_inputs, num_outputs, activ='sigmoid', out_func='sigmoid'):
    shape = [num_inputs] + shape
    for i in range(1, len(shape)):
        in_dim = shape[i-1]
        out_dim = shape[i]
        name = 'hidden_layer' + str(i)
        print('Constructing ' + name + '...')
        network = build_layer(name, input_vec, in_dim, out_dim, activ)
        input_vec = network
    name = 'output_' + out_func
    print('Constructing ' + name + '...')
    network = build_layer(name, input_vec, shape[-1], num_outputs, out_func)
    return network

### Build Loss Function
Returns a loss function based from an output network layer and a function type

In [12]:
def get_loss_function(network_output_layer, labels, loss_func='mse'):
    if loss_func is None or loss_func is '': loss_func = 'mse'
    if loss_func is 'mse': 
        mse = tf.square(network_output_layer - labels)
        loss = tf.reduce_mean(mse, name=loss_func)
        return loss
    elif loss_func is 'xentropy': 
        xent = tf.nn.softmax_cross_entropy_with_logits(network_output_layer, labels, name=loss_func) 
        loss = tf.reduce_mean(xent, name=loss_func)
        return loss
    elif loss_func is 'sig_xentropy':
        sig_xent = tf.nn.sigmoid_cross_entropy_with_logits(network_output_layer, labels, name=loss_func)
        loss = tf.reduce_mean(sig_xent, name=loss_func)
        return loss

### Training Function
Returns a gradient descent training optimizer from a provided loss function for a neural network

In [13]:
def get_training_function(loss_function, learning_rate):
    # Add a scalar summary for the snapshot loss.
    tf.scalar_summary(loss_function.op.name, loss_function)
    # Create the gradient descent optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss_function, global_step=global_step)
    return train_op

### Evaluation
Takes the label data and evaluates the accuracy of the network

In [14]:
def evaluation(network, labels):
    correct = tf.nn.in_top_k(network, labels, 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [15]:
def do_evaluation(sess, network, input_placeholder, label_placeholder, data_set):
    # num correct
    true_count = 0
    steps_per_epoch = data_set.num_test_examples / BATCH_SIZE
    num_examples = steps_per_epoch * BATCH_SIZE
    for step in range(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set, input_placeholder, label_placeholder)
        true_count += sess.run(network, feed_dict=feed_dict)
    precision = true_count / num_examples
    print('Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' % (num_examples, true_count, precision))

## Example: Shape
Build a two-layer relu network with linear softmax output for MNIST data classification
### Constants

In [16]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
INPUT_DIMENSION = 2
OUTPUT_DIMENSION = 1

### Setup

In [17]:
def do_train():
    # Load dataset
    data_set = DataSet(TRAIN_DIR, INPUT_DIMENSION, name='diamond')
    
    with tf.Graph().as_default():
        # Get input and output label placeholders
        input_placeholder, label_placeholder = get_input_label_placeholder(INPUT_DIMENSION)
        
        # Network specification
        net_shape = [128, 32]
        network = construct_network(input_placeholder, net_shape, INPUT_DIMENSION, OUTPUT_DIMENSION)
        
        # Loss function
        loss_function = get_loss_function(network, label_placeholder)
        
        # Training Function
        train_op = get_training_function(loss_function, LEARNING_RATE)
        
        # Evaluation Operation
        eval_op = get_eva

In [18]:
do_train()

/Users/nwolfe/BoxSync/Fall-2015/DeepLearning/HW2/denoising-auto-encoder/data/diamond-train.csv
/Users/nwolfe/BoxSync/Fall-2015/DeepLearning/HW2/denoising-auto-encoder/data/diamond-test.csv
Constructing hidden_layer1...
Constructing hidden_layer2...
Constructing output_sigmoid...
